In [4]:
import torchaudio
torchaudio.set_audio_backend("soundfile")

from datasets.Sampler import Sampler
from graphs.models.VGGVox import VGGVox

from agents.trainer import Trainer


from utils.config import *

# JSON = 'configs/VGGVox.json'
JSON = 'configs/pre_trained/AutoSpeech.json'
# JSON = 'configs/pre_trained/ResNetSE34L.json'
# JSON = 'configs/Test.json'

config = process_config(JSON)

[INFO]: Loading configuration and creating dirs.
[INFO]: Experiment: AutoSpeech Pretrained.
[INFO]: Experiment: AutoSpeech Pretrained.


In [5]:
trainer = Trainer(config)

[INFO]: Operation will be on ***** GPU-CUDA ***** 
[INFO]: Operation will be on ***** GPU-CUDA ***** 
[INFO]: __Python VERSION:  3.9.5 | packaged by conda-forge | (default, Jun 19 2021, 00:22:33) [MSC v.1916 64 bit (AMD64)]
[INFO]: __Python VERSION:  3.9.5 | packaged by conda-forge | (default, Jun 19 2021, 00:22:33) [MSC v.1916 64 bit (AMD64)]
[INFO]: __pyTorch VERSION:  1.9.0
[INFO]: __pyTorch VERSION:  1.9.0
[INFO]: __CUDA VERSION
[INFO]: __CUDA VERSION
[INFO]: __CUDNN VERSION:  7605
[INFO]: __CUDNN VERSION:  7605
[INFO]: __Number CUDA Devices:  1
[INFO]: __Number CUDA Devices:  1
[INFO]: __Devices
[INFO]: __Devices
[INFO]: Active CUDA Device: GPU 0
[INFO]: Active CUDA Device: GPU 0
[INFO]: Available devices  1
[INFO]: Available devices  1
[INFO]: Current cuda device  0
[INFO]: Current cuda device  0


epoch is not in the model.
state_dict is not in the model.
best_acc1 is not in the model.
optimizer is not in the model.
path_helper is not in the model.
genotype is not in the model.


In [6]:
trainer.run()

Epoch 1:   0%|          | 0/37720 [00:00<?, ?it/s]C:\Users\giang\anaconda3\envs\thesis\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Epoch-1 Validation | VEER 50.0000% MDC 1.00000 | Efficiency 92.50% :   0%|          | 136/37720 [00:43<3:20:26,  3.12it/s]
[INFO]: You have entered CTRL+C... Wait to finalize.
[INFO]: You have entered CTRL+C... Wait to finalize.
